Данный файл предназначен для отрисовки различных траекторий, которые нашли алгоритмы COST и TYPES в реализации на C++. Были проведены несколько тестов: в рамках одного теста на одной и той же карте с одинаковыми стартом и финишом искали траектории сначала COST, затем TYPES. Эти результаты находятся в папке `cpp-effective-solution/res` и называны `test<i>-COST.txt` и `test<i>-TYPES.txt` соответственно, где `<i>` -- номер теста.

Кажый файл содержит найденный алгоритмом путь. В случае COST (базовое решение, поиск на state lattice) путём будет послеовательность
дискретных состояний (далее между ними однозначно ищутся примитивы, которые образуют траекторию). В случае TYPES (альтернативно решение
со склеиванием, поиск на графе типов) путём в общем смысле будет последовательность типовых ячеек (= вершины графа типов), чьи проекции образуют коллизионный след траектории (так было в коде на Питоне в папке `Alternative solution`). Однако на пратике можно сильно уменьшить расходуемую память и вместо всех ячеек пути хранить только целевые (в которых заканчивается хоть какой-то примитив) -> именно такие ячейки
сохранены в файле с путём для алгоритма TYPES. Данный код покажет пример, как только по таким ячейкам восстановить всю траекторию.

In [8]:
import sys
import math
import time
from typing import List, Tuple, Optional
from typing_extensions import Self
sys.path.append("../../common/") 
from KC_structs import *
from KC_graphics import *
from KC_searching import *

In [9]:
# загружаем примитивы и типы, которые использовались для поиска траектории:
theta_16 = Theta()  # фиксируем дискретизацию
control_set = ControlSet(theta_16).load_primitives("../cpp-effective-solution/data/main_control_set.txt")
types_set = TypeInfo(control_set).load_types("../cpp-effective-solution/data/main_types.txt")

# считываем карту:
with open("../cpp-effective-solution/maps/Milan_1_256.map", "r") as f:
    map_str = f.read()
task_map = Map().convert_string_to_cells(map_str, obs=True)

R, A = 0, 0  # фиксируем параметры, для которых проводились тесты (все тесты были в точности до целевого состояния, без ошибок по углу/координатам)

### Рисуем результат COST:

In [10]:
def result_COST(i: int):  # i - номер теста
    
    # считываем всё, что нужно, из файла:
    with open(f"../cpp-effective-solution/res/test{i}-COST.txt", "r") as f:
        start = DiscreteState(*map(int, f.readline().split()[-3:]))  # считываем из файла старт и финиш
        finish = DiscreteState(*map(int, f.readline().split()[-3:]))
        assert(f.readline().split()[-1] == "COST")  # проверяем алгоритм
        cost = float(f.readline().split()[-1])  # читаем стоимость пути
        f.readline()  # пропускаем строку
        
        path = []
        while 1:
            str = f.readline()
            if str == "":
                break
            path.append(DiscreteState(*map(int, str.split())))  # добавляем считанные дискртеные состояния на пути (* позволяет распаковать итератор map на отдельные числа, которые подставляются в качестве переменных в DiscreteState)
    path.reverse()  # переворачиваем путь, чтобы он от начала к концу шел

    # рисуем карту, старт и финиш
    ax = draw_task_map(task_map, start, finish, dpi=400, scale=3, A=A, R=R, theta=theta_16)  # увеличиваем масштаб и dpi, чтобы картинка была больше и чётче изображение рисовала
    
    # получаем примитивы между дискретными состояниями и рисуем их -> получаем траекторию:
    col = 'g'
    for k in range(len(path)-1):
        prim = control_set.get_prim_between_states(path[k], path[k+1])  # очередной примитив между соседними вершинами на пути
        col = 'r' if col == 'g' else 'g'  # чередуем цвета: зелёный g и красный r
        ax.plot(prim.x_coords, prim.y_coords, col)  # рисуем его
    
    plt.savefig(f"test{i}-COST__cost={cost:.2f}.png", transparent=False, facecolor='white')  # сохраняем рисунок в файл
    plt.clf()  # удаляем его, чтобы тут не отображался (так как слишком большой)

### Рисуем результат TYPES:

In [11]:
def check_prim(prim: Primitive, task_map: Map) -> bool:
    """
    Функция проверяет, что примитив корректно идёт по карте (не задевает препятсвтия).
    """

    for i, j in zip(prim.collision_in_i, prim.collision_in_j):
        if not (task_map.in_bounds(i,j) and task_map.traversable(i,j)):
            return 0
    return 1


def dfs(start_theta: int, path: List[TypeMesh], final_theta: int) -> List[Primitive]:
    """
    Данная функция должна по path (набору целевых типовых ячеек) восстановить траекторию. Траектория должна 
    начинаться под дискретным углом start_theta и заканчиваться в final_theta. 
    Функция вернёт набор примитивов (уже сдвинутых параллельным перенососм в нужные координаты), которые
    образуют цепочку траектории.
    """

    assert (len(path) > 0)
    
    if len(path) == 1:  # если осталась одна ячейка, путь уже восстановлен
        if theta_16.num_dist(start_theta, final_theta) <= A: 
            return []  # если корректный путь восстановили (по углу), то возвращаем пустой список 
        else:
            return None  # иначе путь не восстановлен

    i0, j0 = path[0].i, path[0].j  # берём первые две целевые типовые ячейки
    i1, j1 = path[1].i, path[1].j
    assert (start_theta in types_set.goal_theta_by_type[path[0].type])  # проверяем, что в path[0] могут начинаться примитивы под углом start_theta
    assert (types_set.goal_theta_by_type[path[1].type] is not None)  # проверяем, что path[1] - целевая
    
    for theta in types_set.goal_theta_by_type[path[1].type]:  # перебираем углы, в которых могут заканчиваться прмиитивы в типово ячейке path[1]
        try:
            prim = control_set.get_prim_between_states(DiscreteState(i0,j0,start_theta), DiscreteState(i1,j1,theta))  # ищем примитив между нужными состояниями
        except:
            continue  # если было исключение (Exception), значит нет такого примитива -> перебираем дальше

        if not check_prim(prim, task_map):  # проверяем, что примитив корректный
            continue
            
        res = dfs(theta, path[1:], final_theta)  # иначе рекурсивно ищем дальше
        if res is not None:
            return [prim] + res  # если нашли дальше, то наращиваем на примитив и возвращаем

    for i in range(2, len(path)):  # не в каждой целевой ячейке обязан кончаться примитив на пути, так как в целевых ячйках могут продолжать какие-то другие примитивы... поэтому стоит так же пропустить некоторые ячейки и искать дальше
        res = dfs(start_theta, [path[0]] + path[i:], final_theta)  # пропускаем 1-ую ячейку и ищем далее
        if res is not None:
            return res
            
    return None  # если перебирали все примитивы и не нашли -> путь не существует (что-то некорректное дали)

In [12]:
def result_TYPES(i: int):  # i - номер теста

    # считываем всё, что нужно, из файла:
    with open(f"../cpp-effective-solution/res/test{i}-TYPES.txt", "r") as f:
        start = DiscreteState(*map(int, f.readline().split()[-3:]))
        finish = DiscreteState(*map(int, f.readline().split()[-3:]))
        assert(f.readline().split()[-1] == "TYPES")
        cost = float(f.readline().split()[-1])
        f.readline()  # пропускаем строку
        
        path = []
        while 1:
            str = f.readline()
            if str == "":
                break
            path.append(TypeMesh(*map(int, str.split())))  # добавляем считанные целевые типовые ячейки на пути
    path.reverse()  # переворачиваем путь

    
    result = dfs(start.theta, path, finish.theta)  # восстанавливаем траекторию
    assert result is not None, "Траектория не восстановилась! Где-то ошибка..."

    # рисуем карту, старт и финиш
    ax = draw_task_map(task_map, start, finish, dpi=400, scale=3, A=A, R=R, theta=theta_16)  
    
    # рисуем найденные примитивы
    col = 'g'
    for prim in result:
        col = 'r' if col == 'g' else 'g'  # чередуем цвета: зелёный g и красный r
        ax.plot(prim.x_coords, prim.y_coords, col) 
    
    for tm in path:  # также рисуем "опорные" целевые ячейки, по которым восстановлена траектория
        ax.add_patch(plt.Rectangle((tm.j-0.5, tm.i-0.5), 1, 1, color='purple', alpha=0.25))
    
    plt.savefig(f"test{i}-TYPES__cost={cost:.2f}.png", transparent=False, facecolor='white')  # сохраняем рисунок в файл
    plt.clf() 
    

### Запускаем отрисовку

In [13]:
for i in range(1, 8):
    result_TYPES(i)
    result_COST(i)

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

...

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>

<Figure size 12000x12000 with 0 Axes>